In [1]:
import os
from tqdm import tqdm as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset

# Initialization

In [2]:
torch.cuda.init()
device = torch.device('cuda:1')
print(torch.cuda.get_device_name(device))

GeForce GTX TITAN X


In [3]:
class AutoEncoder(nn.Module):

    def __init__(self):
        super(AutoEncoder, self).__init__() 
        
        #encode
        self.e1 = nn.Linear(9216,4608)
        self.e2 = nn.Linear(4608,1024)
        
        #decode
        self.d1 = nn.Linear(1024,4608)
        self.d2 = nn.Linear(4608,9216)

    def forward(self, x):
        encode = self.e2(F.relu(self.e1(x)))
        
        return self.d2(F.relu(self.d1(encode)))

# class AutoEncoder(nn.Module):

#     def __init__(self):
#         super(AutoEncoder, self).__init__() 
        
#         #encode
#         self.e1 = nn.Linear(9216,6000)
#         self.e2 = nn.Linear(6000,4000)
# #         self.e3 = nn.Linear(4000,2000)
# #         self.e4 = nn.Linear(2000,1000)
        
#         #decode
# #         self.d1 = nn.Linear(1000,2000)
# #         self.d3 = nn.Linear(2000,4000)
#         self.d1 = nn.Linear(4000,6000)
#         self.d2 = nn.Linear(6000,9216)

#     def forward(self, x):
#         encode = self.e2(F.relu(self.e1(x)))
#         return self.d2(F.relu(self.d1(encode)))

In [4]:
base_path = '/ssd2/chetanp'
batch_size = int((2 ** 14))

In [5]:
criterion = nn.CosineSimilarity()

loss_function = lambda x,y: (1-criterion(x,y)).mean()

# loss_function = nn.MSELoss()

In [6]:
model = nn.DataParallel(AutoEncoder(),device_ids =[1,2]).to(device)
path = os.path.join(base_path,'brown_e_base_compressor_superfine.pth')
model.load_state_dict(torch.load(path))
model.eval()

DataParallel(
  (module): AutoEncoder(
    (e1): Linear(in_features=9216, out_features=4608, bias=True)
    (e2): Linear(in_features=4608, out_features=1024, bias=True)
    (d1): Linear(in_features=1024, out_features=4608, bias=True)
    (d2): Linear(in_features=4608, out_features=9216, bias=True)
  )
)

# Testing The Model

In [7]:
dataset = []
for i in range(2):
    path = os.path.join(base_path,'brown_e_base'+str(i+4)+'.pt')
    data = torch.load(path)
    dataset.append(TensorDataset(data))
dataset = ConcatDataset(dataset)
data_loader = DataLoader(dataset,batch_size = batch_size,num_workers = 14)

In [8]:
running_loss = 0
count = 0
    
for i,data in tqdm(enumerate(data_loader)):
    ## Get Data
    sample = data[0].to(device)
        
    ## Pass forward
    output = model(sample)
    loss = loss_function(output,sample)
      
    ## Update Running Loss
    running_loss += float(loss)
    count += 1
    
#     if i == 0:
#         diff = sample-output
#         value = torch.sum(diff.pow(2), dim = 1).mean()
#         print(value)
print('Test Set Loss: ', str(running_loss/count))

17it [00:16,  1.49it/s]

Test Set Loss:  0.04370893998181119


In [9]:
dataset = []
for i in range(6):
    path = os.path.join(base_path,'brown_e_base'+str(i)+'.pt')
    data = torch.load(path)
    dataset.append(TensorDataset(data))
dataset = ConcatDataset(dataset)
data_loader = DataLoader(dataset,batch_size = batch_size,num_workers = 14)

In [10]:
running_loss = 0
count = 0
    
for data in tqdm(data_loader):
    ## Get Data
    sample = data[0].to(device)
        
    ## Pass forward
    output = model(sample)
    loss = loss_function(output,sample)
      
    ## Update Running Loss
    running_loss += float(loss)
    count += 1
        
print('Overall Loss: ', str(running_loss/count))

100%|██████████| 71/71 [00:59<00:00,  1.46it/s]

Overall Loss:  0.03977823509296901


In [11]:
dataset = []
for i in range(4):
    path = os.path.join(base_path,'brown_e_base'+str(i)+'.pt')
    data = torch.load(path)
    dataset.append(TensorDataset(data))
dataset = ConcatDataset(dataset)
data_loader = DataLoader(dataset,batch_size = batch_size,num_workers = 14)

In [12]:
running_loss = 0
count = 0
    
for data in tqdm(data_loader):
    ## Get Data
    sample = data[0].to(device)
        
    ## Pass forward
    output = model(sample)
    loss = loss_function(output,sample)
      
    ## Update Running Loss
    running_loss += float(loss)
    count += 1
        
print('Training Loss: ', str(running_loss/count))

100%|██████████| 55/55 [00:46<00:00,  1.70it/s]

Training Loss:  0.03854962655089118
